## 데이터 파이프라인 End-to-End 프로젝트

<img src="https://velog.velcdn.com/images/newnew_daddy/post/7422a14f-8952-4990-9aba-578695cce6f7/image.png" width="60%">

1. postgresql DB에 Fake 유저 정보 저장
2. Pipeline 모듈을 통한 데이터 이행 및 데이터 가공
3. MYSQL DB에 가공된 테이블 저장
4. Streamlit 라이브러리를 통한 MYSQL 테이블 데이터 시각화
5. n초 마다 Batch성 이행을 통해 시각화 테이블 업데이트 내용 확인

### 1. PostgreSQL DB에 Fake 유저 정보 저장

##### 1) Fake 데이터 생성

Faker 라이브러리
- [공식 Docs](https://faker.readthedocs.io/en/master/)
- [Fake data 사용하기](https://www.daleseo.com/python-faker/)

In [1]:
# Fake data

from faker import Faker

fake = Faker("ko_KR")
fake.profile()

{'job': '기타 문리/기술 및 예능 강사',
 'company': '김김김',
 'ssn': '240018-2142689',
 'residence': '대구광역시 강북구 반포대898로 (지은최면)',
 'current_location': (Decimal('9.630469'), Decimal('105.661162')),
 'blood_group': 'O-',
 'website': ['http://yuhanhoesa.com/', 'http://icoe.kr/'],
 'username': 'gimhyeonju',
 'name': '이현지',
 'sex': 'F',
 'address': '대구광역시 서구 학동로',
 'mail': 'omyeongja@nate.com',
 'birthdate': datetime.date(2023, 10, 1)}

In [ ]:
# 데이터 가공
import shortuuid


key_list = ['name', 'ssn', 'job', 'residence', 'blood_group', 'sex', 'birthdate']
user_data = dict()

for key in key_list:
    user_data[key] = fake.profile()[key]
    
    if key == 'birthdate':
        user_data[key] = str(fake.profile()[key].year) + str(fake.profile()[key].month).zfill(2) + str(fake.profile()[key].day).zfill(2)

user_data['uuid'] = shortuuid.uuid()

In [ ]:
# 함수 형태로 생성

from faker import Faker
import shortuuid

key_list = ['name', 'job', 'residence', 'blood_group', 'sex', 'birthdate']

def create_fake_user(key_list):
    
    fake = Faker("ko_KR")
    user_data = dict()

    for key in key_list:
        user_data[key] = fake.profile()[key]
        
        if key == 'birthdate':
            user_data[key] = str(fake.profile()[key].year) + str(fake.profile()[key].month).zfill(2) + str(fake.profile()[key].day).zfill(2)

    user_data['uuid'] = shortuuid.uuid()
    
    return user_data

##### 2) PostgreSQL DB 테이블 생성

In [2]:
import psycopg2
import pandas as pd

# database connection 생성
db = psycopg2.connect(
    host='localhost',
    dbname='postgres',
    user='hyunsoo',
    password='150808',
    port=5432
    )

cursor=db.cursor()

pd.read_sql("SELECT * FROM fake_user", db)

C:\Users\YEIN\AppData\Local\Temp\ipykernel_16976\3625970781.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM fake_user", db)


,uuid,name,job,residenct,blood_group,sex,birthdate
0,hiHBkVffEpBQYjnzHx3abs,조시우,웹 및 멀티미디어 디자이너,인천광역시 서초구 영동대87로,AB-,F,19190212
1,agpsZF4GY7BNvG93eeyvk2,이민준,기타 비금속제품관련 생산기 조작원,대구광역시 용산구 역삼387길,AB+,F,19811230
2,7ADnduVUeGHmsbPP2zYiBW,최영호,식품공학 기술자 및 연구원,울산광역시 금천구 삼성가,AB-,M,20040127
3,E45xDDdVZejWnmUzSNgDz5,김상호,화학제품 생산기 조작원,부산광역시 강북구 서초대가 (지아문김리),AB-,M,19420603
4,VfAHfJZGK4nkLFhfh8Jcxg,서숙자,기타 음식서비스 종사원,전라북도 수원시 권선구 반포대길,O+,F,19271101
...,...,...,...,...,...,...,...
190,A7zm7EaPQEmh4LWXGxmEDd,김영식,기타 스포츠 및 레크레이션 관련 전문가,대구광역시 송파구 봉은사0로,A+,F,20000522
191,dUMNKWcjqSTRK74iLJMqAQ,박미경,화학물 가공장치 조작원,경상북도 화천군 도산대07로,AB+,M,19591206
192,CALheW57s9ygAq9D393Eeo,안영미,자동차 부분품 조립원,경상북도 단양군 봉은사943가,O+,M,20210120
193,hjYiBAKAHZ5xMDd47auPz3,양예진,기술 및 기능계 강사,경상북도 군포시 도산대352가,A+,M,19150819


In [2]:
create_table = """
CREATE TABLE fake_user
(
    uuid VARCHAR(50) PRIMARY KEY,
    name VARCHAR(10),
    job VARCHAR(50),
    residenct VARCHAR(50),
    blood_group VARCHAR(10),
    sex VARCHAR(10),
    birthdate VARCHAR(10)
);
"""

In [3]:
cursor.execute(create_table)
db.commit()

##### 3) Fake data Insert - 예시

In [24]:
cursor=db.cursor()
userkey_list = ['uuid', 'name', 'job', 'residence', 'blood_group', 'sex', 'birthdate']

for _ in range(5):
    user_value = tuple([create_fake_user()[key] for key in userkey_list])
    cursor.execute(f"INSERT INTO fake_user VALUES{user_value}")
    
db.commit()

##### 4) 함수 생성

In [ ]:
from db.connector import DBconnector
from settings import DB_SETTINGS
from psycopg2.extras import execute_values

def insert_data():
    
    db_connector = DBconnector(**DB_SETTINGS['POSTGRES'])

    with db_connector as connected:
        
        cursor=connected.conn.cursor()
        userkey_list = ['uuid', 'name', 'job', 'residence', 'blood_group', 'sex', 'birthdate']
        
        insert_list = [tuple([create_fake_user()[key] for key in userkey_list]) for _ in range(7)]
        print(insert_list)
        
        sql = f"INSERT INTO fake_user2 VALUES %s;"
        execute_values(cursor, sql, insert_list)
            
        connected.conn.commit()

In [ ]:
import random

def insert_data():
    
    db_connector = DBconnector(**DB_SETTINGS['POSTGRES'])

    with db_connector as connected:
        
        cursor=connected.conn.cursor()
        userkey_list = ['uuid', 'name', 'job', 'residence', 'blood_group', 'sex', 'birthdate']
        row_add = random.randint(5,15)
        
        insert_list = [tuple([create_fake_user()[key] for key in userkey_list]) for _ in range(row_add)]
        
        sql = f"INSERT INTO fake_user2 VALUES %s;"
        execute_values(cursor, sql, insert_list)
            
        connected.conn.commit()

### 2. 모듈을 통한 데이터 이행 및 데이터 가공

##### 1) 데이터 가공
- Pandas를 통해 Data Mart 형태의 테이블로 가공
  1. 거주하는 도시 통계 -> `residence` 전처리 및 컬럼 생성
  2. 혈액형 통계 -> `blood_group`	 전처리 및 컬럼 생성
  3. 남녀 통계 -> `sex` 컬럼 사용
  4. 나이대 통계 -> `birthdate` 전처리 및 컬럼 생성

In [1]:
import pandas as pd
import psycopg2

# database connection 생성
db = psycopg2.connect(host='localhost', dbname='postgres', user='hyunsoo', password='150808', port=5432)

pd.read_sql("SELECT * FROM fake_user LIMIT 10", db)

In [ ]:
df = pd.read_sql("SELECT * FROM fake_user LIMIT 10", db)

In [35]:
# 도시 컬럼 생성

df['city'] = df['residenct'].str.split().str[0]

df.head()

,uuid,name,ssn,job,residenct,blood_group,sex,birthdate,city
0,WbA7aKjcFkwEaWy94xavJK,김미경,310011-1308197,경영 및 진단 전문가,광주광역시 서대문구 영동대가 (지아김이동),AB+,F,19540305,광주광역시
1,9V8oDsu6rY2xaFL2PpoFZi,이서영,050626-1443817,치과 의사,경기도 서산시 압구정71길 (정웅김리),A-,F,20180507,경기도
2,K2q362jgV3HQBZ7my2Wnj3,백정자,940208-1044850,영업 및 판매 관련 관리자,경상남도 청주시 흥덕구 도산대로,B-,F,19380724,경상남도
3,KC9YHiZxcwmNMnSZQ37U8d,장상호,850521-1526902,기타 식품가공관련 종사원,전라남도 횡성군 백제고분709가,B-,M,20110824,전라남도
4,24e3E64Ci5G63AH2TQB5KH,김지아,240403-2387302,주방 보조원,울산광역시 송파구 삼성가 (수민박리),B+,M,19971131,울산광역시


In [21]:
# 출생년도 컬럼 생성

df['birthdate'] = df['birthdate'].astype(str)

df['birthyear'] = df['birthdate'].str.slice(0,4)

df.head()

,uuid,name,ssn,job,residenct,blood_group,sex,birthdate,city,birthyear
0,WbA7aKjcFkwEaWy94xavJK,김미경,310011-1308197,경영 및 진단 전문가,광주광역시 서대문구 영동대가 (지아김이동),AB+,F,19540305,광주광역시,1954
1,9V8oDsu6rY2xaFL2PpoFZi,이서영,050626-1443817,치과 의사,경기도 서산시 압구정71길 (정웅김리),A-,F,20180507,경기도,2018
2,K2q362jgV3HQBZ7my2Wnj3,백정자,940208-1044850,영업 및 판매 관련 관리자,경상남도 청주시 흥덕구 도산대로,B-,F,19380724,경상남도,1938
3,KC9YHiZxcwmNMnSZQ37U8d,장상호,850521-1526902,기타 식품가공관련 종사원,전라남도 횡성군 백제고분709가,B-,M,20110824,전라남도,2011
4,24e3E64Ci5G63AH2TQB5KH,김지아,240403-2387302,주방 보조원,울산광역시 송파구 삼성가 (수민박리),B+,M,19971131,울산광역시,1997


In [26]:
# 나이 컬럼 생성

current_year = 2024
df['age'] = current_year - df['birthyear'].astype(int)

# def categorize_age(age):
#     if 0 <= age < 10:
#         return '0대'
#     elif 10 <= age < 20:
#         return '10대'
#     elif 20 <= age < 30:
#         return '20대'
#     elif 30 <= age < 40:
#         return '30대'
#     elif 40 <= age < 50:
#         return '40대'
#     elif 50 <= age < 60:
#         return '50대'
#     elif 60 <= age < 70:
#         return '60대'
#     elif 70 <= age < 80:
#         return '70대'
#     elif 80 <= age < 90:
#         return '80대'
#     else:
#         return '90대 이상'

def categorize_age(age):
    if 0 <= age < 10:
        return '0대'
    else:
        if age >= 90:
            return "80대 이상"
        else:
            age_group = (age) // 10 * 10
            return f'{age_group}대'

# Apply the categorize_age function to create a new column 'age_category'
df['age_category'] = df['age'].apply(categorize_age)

df.head()

,uuid,name,ssn,job,residenct,blood_group,sex,birthdate,city,birthyear,age,age_category
0,WbA7aKjcFkwEaWy94xavJK,김미경,310011-1308197,경영 및 진단 전문가,광주광역시 서대문구 영동대가 (지아김이동),AB+,F,19540305,광주광역시,1954,70,70대
1,9V8oDsu6rY2xaFL2PpoFZi,이서영,050626-1443817,치과 의사,경기도 서산시 압구정71길 (정웅김리),A-,F,20180507,경기도,2018,6,0대
2,K2q362jgV3HQBZ7my2Wnj3,백정자,940208-1044850,영업 및 판매 관련 관리자,경상남도 청주시 흥덕구 도산대로,B-,F,19380724,경상남도,1938,86,80대
3,KC9YHiZxcwmNMnSZQ37U8d,장상호,850521-1526902,기타 식품가공관련 종사원,전라남도 횡성군 백제고분709가,B-,M,20110824,전라남도,2011,13,10대
4,24e3E64Ci5G63AH2TQB5KH,김지아,240403-2387302,주방 보조원,울산광역시 송파구 삼성가 (수민박리),B+,M,19971131,울산광역시,1997,27,20대


In [31]:
set(df['blood_group'].to_list())

df['blood'] = df['blood_group'].str.slice(0, -1)

df.head()

,uuid,name,ssn,job,residenct,blood_group,sex,birthdate,city,birthyear,age,age_category,blood
0,WbA7aKjcFkwEaWy94xavJK,김미경,310011-1308197,경영 및 진단 전문가,광주광역시 서대문구 영동대가 (지아김이동),AB+,F,19540305,광주광역시,1954,70,70대,AB
1,9V8oDsu6rY2xaFL2PpoFZi,이서영,050626-1443817,치과 의사,경기도 서산시 압구정71길 (정웅김리),A-,F,20180507,경기도,2018,6,0대,A
2,K2q362jgV3HQBZ7my2Wnj3,백정자,940208-1044850,영업 및 판매 관련 관리자,경상남도 청주시 흥덕구 도산대로,B-,F,19380724,경상남도,1938,86,80대,B
3,KC9YHiZxcwmNMnSZQ37U8d,장상호,850521-1526902,기타 식품가공관련 종사원,전라남도 횡성군 백제고분709가,B-,M,20110824,전라남도,2011,13,10대,B
4,24e3E64Ci5G63AH2TQB5KH,김지아,240403-2387302,주방 보조원,울산광역시 송파구 삼성가 (수민박리),B+,M,19971131,울산광역시,1997,27,20대,B


In [33]:
pdf = df[['uuid', 'name', 'job', 'sex', 'city', 'birthyear', 'age', 'age_category', 'blood']]

pdf.head()

,uuid,name,job,sex,city,birthyear,age,age_category,blood
0,WbA7aKjcFkwEaWy94xavJK,김미경,경영 및 진단 전문가,F,광주광역시,1954,70,70대,AB
1,9V8oDsu6rY2xaFL2PpoFZi,이서영,치과 의사,F,경기도,2018,6,0대,A
2,K2q362jgV3HQBZ7my2Wnj3,백정자,영업 및 판매 관련 관리자,F,경상남도,1938,86,80대,B
3,KC9YHiZxcwmNMnSZQ37U8d,장상호,기타 식품가공관련 종사원,M,전라남도,2011,13,10대,B
4,24e3E64Ci5G63AH2TQB5KH,김지아,주방 보조원,M,울산광역시,1997,27,20대,B


##### 2) 데이터 가공 함수 생성

In [ ]:
def transform_data(pandas_df):
    current_year = 2024
    
    # 도시 컬럼 생성
    pandas_df['city'] = pandas_df['residenct'].str.split().str[0]
    
    # 출생년도 컬럼 생성
    pandas_df['birthdate'] = pandas_df['birthdate'].astype(str)
    pandas_df['birthyear'] = pandas_df['birthdate'].str.slice(0,4)
    
    # 혈액형 컬럼 생성
    pandas_df['blood'] = pandas_df['blood_group'].str.slice(0, -1)
    
    # 나이 컬럼 생성
    pandas_df['age'] = current_year - pandas_df['birthyear'].astype(int)

    # 연령대 컬럼 생성
    pandas_df['age_category'] = pandas_df['age'].apply(categorize_age)
    
    # 필요한 컬럼만 선택
    df = pandas_df[['uuid', 'name', 'job', 'sex', 'city', 'birthyear', 'age', 'age_category', 'blood']]

    return df
    
    
def categorize_age(age):
    if 0 <= age < 10:
        return '0대'
    else:
        if age >= 90:
            return "80대 이상"
        else:
            age_group = (age) // 10 * 10
            return f'{age_group}대'

In [ ]:
transform_data(df).head()

##### 3) 데이터 이행 모듈에 통합

In [1]:
from db.connector import DBconnector
from db.postgresql_query import queries
from settings import DB_SETTINGS, TEMP_PATH
from pipeline.extract import extractor
from pipeline.data_transform import transformer
from pipeline.load import loader

In [ ]:
batch_date = "20240202"

def main(batch_date):
    
    db_connector = DBconnector(**DB_SETTINGS['POSTGRES'])
    
    for table_name in queries:
        print(table_name)
        pandas_df = extractor(db_connector, table_name, batch_date)
        
        res, df = transformer(batch_date, table_name, pandas_df)
        
        db_connector = DBconnector(**DB_SETTINGS['MYSQL'])
        
        loader(db_connector, df, table_name)
            
main(batch_date)
    

### 3. start.py에 반영
- 특정 간격(3초, 5초 등) 기준으로 fake data가 Insert 될 수 있도록 설정

In [ ]:
from datetime import datetime, timedelta, timezone
import click, time
from pipeline import controller

from fake_data.insert import insert_data

@click.command
@click.option('-d', '--custom-batch-date', type=click.STRING, default='', help='배치작업연월일')

def start_batch(custom_batch_date):
    
    for _ in range(10):
        print("START BATCH")
        insert_data()
        
        if custom_batch_date:
            batch_date = custom_batch_date
        else:
            batch_date = (datetime.now(timezone(timedelta(hours=9))) - timedelta(days = 1)).strftime('%Y%m%d')
            
        controller.main(batch_date)
        time.sleep(5)
        
if __name__ == "__main__":
    
    start_batch()

### 4. 시각화 코드 작성 (streamlit)

> 웹페이지 시작 : streamlit run [시각화 Python 파일]

In [ ]:
import streamlit as st
import pandas as pd

from db.connector import DBconnector
from settings import DB_SETTINGS

## MYSQL DB에서 DataMart 데이터 읽기
db_connector = DBconnector(**DB_SETTINGS['MYSQL'])
with db_connector as connected:
    
    df = pd.read_sql("SELECT * FROM fake_datamart;", connected.conn)

## 차트에 사용할 데이터 생성
df_sex = df.groupby('sex').size().reset_index(name='count')
df_blood = df.groupby('blood').size().reset_index(name='count')
df_city = df.groupby('city').size().reset_index(name='count')
df_age = df.groupby('age_category').size().reset_index(name='count')

## 차트 그리기
# Row Count
st.title(f'TOTAL DATA : {len(df)}')
st.dataframe(df.head())

# 성별
st.header('SEX GROUP BY')
st.bar_chart(df_sex, x='sex', y='count')

# 혈액형
st.header('BLOOD TYPE GROUP BY')
st.bar_chart(df_blood, x='blood', y='count')

# 거주 도시
st.header('CITY GROUP BY')
st.bar_chart(df_city, x='city', y='count')

# 나이
st.header('AGE GROUP BY')
st.line_chart(df_age, x='age_category', y='count')